<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_analysis/News_Classification_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install tensorflow

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x3a1a6000 @  0x7fa592f99615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 21kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588507 sha256=8f83007cae48ed40bf6cee36f2d06bb5f6b62e7e7afd3ccac80b9cac4a0e193c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 8.5MB

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-zzbtvasq
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zzbtvasq
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=e134f08b4ed6bd4fae72315bb328f894efca01fdb8e692404460f177828a197f
  Stored in directory: /tmp/pip-ephem-wheel-cache-60cb1cuv/wheels/43/7b/50/91bc4d7bebb30b62f31a30a537787eb512f27506909162ba14
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

In [4]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
#GPU 사용 시
device = torch.device("cuda:0")

# **SKT의 KoBERT 이용**

## **데이터 불러오기**

In [6]:
train_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_accomodation_train.csv'
test_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_accomodation_test.csv'

In [7]:
train_data = pd.read_csv(train_path, encoding='utf-8', index_col=0)
test_data = pd.read_csv(test_path, encoding='utf-8', index_col=0)
len(train_data), len(test_data)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1998550, 499637)

In [8]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(1998550, 499637)

In [9]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['content'][i], int(train_data['label'][i])]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['content'][i], int(test_data['label'][i])])

dataset_test[:5], dataset_train[:5]

100%|██████████| 499637/499637 [00:14<00:00, 34843.20it/s]


([['/문호남 기자 munonam@  택시업계와 정부, 업계가 지난한 과정 끝에 처음으로 협상 테이블에 앉았다.', 0],
  ["이미 승차공유 이용자 모임인 '카풀러'가 회원수 2만명을 돌파했음에도 불구하고 대타협기구의 의사결정단계에서 이들의 의견을 수렴하는 통로는 없었다.",
   0],
  ['업계 관계자는 이제는 진짜 공유경제를 논하는 생산적인 자리가 마련돼야 한다 며 택시기사들의 처우를 개선하고 시대에 맞지 않는 택시 규제는 개선하면서 카풀과도 얽힌 실타래를 풀 수 있는 방안을 진지하게 고민해야 할 것 이라고 했다.',
   0],
  ['손흥민은 이날 후반 34분 요렌테가 가슴으로 떨궈준 공을 오른발 슈팅으로 뉴캐슬의 골망을 흔들었다.', 0],
  ['2경기 연속 골에 시즌 14호골, 리그 10호골이었다.', 0]],
 [['그러면서 음모론의 문제는 현실감각을 마비시켜 버린다는 데에 있다.', 0],
  ['이 자리는 2019학년도에 추진될 IPP형 일학습병행의 운영을 위해 기업과 학습근로자가 교감하는 시간으로 마련됐다.', 1],
  ['행정 처분이 별로 없어서 모든 문제를 검찰이 판단해야 한다.', 0],
  ['민생침해팀은 는 최근 전국 유흥업소 21곳에 대해 동시 세무조사에 착수해 큰 성과를 거뒀다.', 0],
  ['노사정 논의에 참여않은채 탄력근로제 반대에만 몰입 지지 못얻는 총파업 되풀이 노동 新패러다임 짜기 바쁜데 강경파에 묻혀 관련논의 실종 ◆ 미래대비 눈감은 민노총 ◆  노동인권실현을위한노무사모임, 민주사회를위한변호사모임, 민주노총·금속노조 법률원 등이 5일 청와대 앞에서 기자회견을 열고 경제사회노동위원회의 탄력근로제 단위기간 확대 합의를 철회하고, 국제노동기구 핵심 협약을 조건 없이 비준하라고 촉구하고 있다.',
   0]])

## **데이터 토큰화**

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1  # 테스트용이기에 에폭은 1만 지정하겠습니다.
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## **모델링**

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 해당 부분 파라미터 조정으로 다중 분류 가능
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [20]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [21]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total) # Warm-Up Step 추가

In [22]:
def calc_accuracy(X,Y): # 정확도 계산 함수
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## **모델 학습**

In [23]:
PATH = "/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/분석 모델/model_checkpoint/숙박업/{}_epochs".format(num_epochs) # 모델 저장 경로 설정

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 모델 학습 : model.train()을 지정해줘야 Fine Tunning이 이뤄짐
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 평가 : model.eval()을 지정해야 Fine Tunning을 멈추고 평가를 시작함
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # 에폭마다 체크포인트 저장
    torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }, PATH + 'all.tar')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


epoch 1 batch id 1 loss 0.6869012713432312 train acc 0.546875
epoch 1 batch id 201 loss 0.7017337083816528 train acc 0.5002332089552238
epoch 1 batch id 401 loss 0.7021018862724304 train acc 0.514300187032419
epoch 1 batch id 601 loss 0.6885784864425659 train acc 0.5177568635607321


## **모델링 결과 시각화**